## Project 3
This will analyze route data in supply chain with ML, and find the optimal route

In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import folium
import pickle
import matplotlib.pyplot as plt
#import contextily as ctx
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import networkx as nx
import osmnx as ox
import numpy as np

In [2]:
# Load the dataset into a pandas dataframe
df = pd.read_csv('deliverytime.csv')

In [3]:
# Confirm the dataset
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Type_of_order,Type_of_vehicle,Time_taken(min)
0,4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,Snack,motorcycle,24
1,B379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,Snack,scooter,33
2,5D6D,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,Drinks,motorcycle,26
3,7A6A,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,Buffet,motorcycle,21
4,70A2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,Snack,scooter,30


In [4]:
# Data cleaning: Drop unnecessary columns
df = df.drop(columns=['ID','Delivery_person_ID','Delivery_person_Age','Delivery_person_Ratings','Type_of_order','Type_of_vehicle'])

In [5]:
# Reconfirm data
df.head()

,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Time_taken(min)
0,22.745049,75.892471,22.765049,75.912471,24
1,12.913041,77.683237,13.043041,77.813237,33
2,12.914264,77.678400,12.924264,77.688400,26
3,11.003669,76.976494,11.053669,77.026494,21
4,12.972793,80.249982,13.012793,80.289982,30


In [6]:
# Rename column names
df = df.rename(columns={'Restaurant_latitude':'start_lon' ,'Restaurant_longitude': 'start_lat', 'Delivery_location_latitude':'end_lon','Delivery_location_longitude':'end_lat','Time_taken(min)':'travel_time'})

# Display df
df.head()

,start_lon,start_lat,end_lon,end_lat,travel_time
0,22.745049,75.892471,22.765049,75.912471,24
1,12.913041,77.683237,13.043041,77.813237,33
2,12.914264,77.678400,12.924264,77.688400,26
3,11.003669,76.976494,11.053669,77.026494,21
4,12.972793,80.249982,13.012793,80.289982,30


## Run Machine Learning Model

In [7]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df[['start_lon', 'start_lat', 'end_lon', 'end_lat']], df['travel_time'], test_size=0.2)

# Create a Random Forest regressor model
rf = RandomForestRegressor()

# Train the model on the training set
rf.fit(X_train, y_train)


RandomForestRegressor()

In [8]:
# Use the trained model to make predictions on the testing set
y_pred = rf.predict(X_test)

# Calculate the mean squared error of the predictions
mse = mean_squared_error(y_test, y_pred)

print(f"Mean squared error: {mse:.2f}")

Mean squared error: 85.99


In [9]:
# Save the model to a .pkl file
with open('transportation_route_model.pkl', 'wb') as f:
    pickle.dump(rf, f)

In [14]:
# Take sample data based on the original dataset
sample_df = df.head(20)

## Front End
This function takes from to location information, and calculate time based on machine learning model with RandomeForestRegressor

In [10]:
# This function takes from to location information, and calculate time

import pickle
import numpy as np

# Load the machine learning model from file
with open('transportation_route_model.pkl', 'rb') as f:
    model = pickle.load(f)

def predict_travel_time(start_lon, start_lat, end_lon, end_lat):
    # Create an array with the input features
    X = np.array([[start_lon, start_lat, end_lon, end_lat]])

    # Load the trained machine learning model
    with open('transportation_route_model.pkl', 'rb') as f:
        model = pickle.load(f)

    # Make a prediction using the input features
    travel_time = model.predict(X)

    # Return the predicted travel time
    return travel_time[0]

In [65]:
# Prompt the user for input
# 22.745049	75.892471	22.765049	75.912471	
start_lon = float(input("Enter the starting longitude: "))
start_lat = float(input("Enter the starting latitude: "))
end_lon = float(input("Enter the ending longitude: "))
end_lat = float(input("Enter the ending latitude: "))

# Call the predict_travel_time() function with the user input
travel_time = predict_travel_time(start_lon, start_lat, end_lon, end_lat)

# Print the estimated travel time
print(f"The estimated travel time is {travel_time:.2f} minutes.")

Enter the starting longitude: 22.745049
Enter the starting latitude: 75.892471
Enter the ending longitude: 22.765049
Enter the ending latitude: 75.912471
The estimated travel time is 22.08 minutes.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [17]:
# N/A
# This function takes address and give you longitute and latitude information

from geopy.geocoders import Nominatim

def address_to_coordinates():
    # Initialize the Nominatim geolocator
    geolocator = Nominatim(user_agent="my_app")

    # Prompt the user for an address
    address = input("Enter an address: ")

    # Use the geolocator to get the latitude and longitude coordinates of the address
    location = geolocator.geocode(address)

    if location is None:
        print(f"Sorry, could not find a location for {address}")
    else:
        # Print the latitude and longitude coordinates of the address
        print(f"The latitude and longitude coordinates of {address} are: {location.latitude}, {location.longitude}")


In [18]:
address_to_coordinates()

Enter an address:  171 Bleecker St Toronto ON


The latitude and longitude coordinates of 171 Bleecker St Toronto ON are: 43.66700414285714, -79.37352042857144


## This function takes from and to address and tell you how long it takes

In [11]:
# This function takes from and to address and tell you how long it takes

from geopy.geocoders import Nominatim

def estimate_time_calculator(address_from, address_to):
    # Initialize the Nominatim geolocator
    geolocator = Nominatim(user_agent="my_app")

    # Use the geolocator to get the latitude and longitude coordinates of the address
    location_from = geolocator.geocode(address_from)
    location_to = geolocator.geocode(address_to)

    if location_from is None or location_to is None:
        return None

    # Call the predict_travel_time() function with the user input
    travel_time = predict_travel_time(location_from.longitude, location_from.latitude, location_to.longitude, location_to.latitude)

    return travel_time


#def estimate_time_calculator():
#    # Initialize the Nominatim geolocator
#    geolocator = Nominatim(user_agent="my_app")#

    # Prompt the user for an address
#    address_from = input("Enter a from address: ")
#    address_to = input("Enter a to address: ")

    # Use the geolocator to get the latitude and longitude coordinates of the address
#    location_from = geolocator.geocode(address_from)
#    location_to = geolocator.geocode(address_to)

#    if location_from is None:
#        print(f"Sorry, could not find a location for {address_from}")
#    else:
#        # Print the latitude and longitude coordinates of the address
#        print(f"The latitude and longitude coordinates of {address_from} are: {location_from.latitude}, {location_from.longitude}")
#        print(f"The latitude and longitude coordinates of {address_to} are: {location_to.latitude}, {location_to.longitude}")

#        # Call the predict_travel_time() function with the user input
#        travel_time = predict_travel_time(location_from.longitude, location_from.latitude, location_to.longitude, location_to.latitude)

        # Print the estimated travel time
#        print(f"The estimated travel time is {travel_time:.2f} minutes.")

In [15]:
estimate_time_calculator()

Enter a from address:  100 Stokes Street Toronto Ontario
Enter a to address:  2329 West Mall Vancouver, BC Canada


The latitude and longitude coordinates of 100 Stokes Street Toronto Ontario are: 43.643352750000005, -79.41914837266222
The latitude and longitude coordinates of 2329 West Mall Vancouver, BC Canada are: 49.2606569, -123.2534078
The estimated travel time is 23.98 minutes.


C:\Users\keito\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"


In [12]:
address_to_coordinates()

Enter a from address:  100 Stokes Street Toronto Ontario
Enter a to address:  2329 West Mall Vancouver, BC Canada


The latitude and longitude coordinates of 100 Stokes Street Toronto Ontario are: 43.643352750000005, -79.41914837266222
The latitude and longitude coordinates of 2329 West Mall Vancouver, BC Canada are: 49.2606569, -123.2534078
The estimated travel time is 23.98 minutes.


C:\Users\keito\anaconda3\envs\dev\lib\site-packages\sklearn\base.py:451: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  "X does not have valid feature names, but"


In [12]:
import sys
print(sys.path)

['C:\\Users\\keito\\uoft_fintech\\project-3\\machine_learning', 'C:\\Users\\keito\\anaconda3\\envs\\dev\\python37.zip', 'C:\\Users\\keito\\anaconda3\\envs\\dev\\DLLs', 'C:\\Users\\keito\\anaconda3\\envs\\dev\\lib', 'C:\\Users\\keito\\anaconda3\\envs\\dev', '', 'C:\\Users\\keito\\anaconda3\\envs\\dev\\lib\\site-packages', 'C:\\Users\\keito\\anaconda3\\envs\\dev\\lib\\site-packages\\win32', 'C:\\Users\\keito\\anaconda3\\envs\\dev\\lib\\site-packages\\win32\\lib', 'C:\\Users\\keito\\anaconda3\\envs\\dev\\lib\\site-packages\\Pythonwin', 'C:\\Users\\keito\\anaconda3\\envs\\dev\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\keito\\.ipython']
